In [1]:
import os
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
from numpy import sqrt
import networkx as nx
from sklearn.linear_model import Lasso
from page import pagerank
# from Bert_function import author_embedding
from deepwalk_generate import DpWalk
from processing_abstract import process_abstracts
from process_authorFile import process_authorFiles
from final_dico_creation import dictionary_concatenation
from final_dico_creation import dictionary_concatenation2  # keeps abstracts separated
from Text_Embedding2 import Embed_Author
import pickle
import copy
from MLP import prepare_data, MLP, train_model, evaluate_model, predict
import nltk
print("Imported all libraries.")


Imported all libraries.


In [ ]:
## don't run ##
print("Loading & printing DictForAuthor_new.pkl")
infile = open('DictForAuthor_new.pkl', 'rb')
DictForAuthor_new = pickle.load(infile)
infile.close()

print("type(DictForAuthor_new) : ", type(DictForAuthor_new))
print("len(DictForAuthor_new) = ", len(DictForAuthor_new))
# print("sentences[0] : \n", sentences[0])
print("DictForAuthor_new.pkl is compromised. ")


In [ ]:
## don't run ##
''' 
To generate a dictionary matching the authors to their concatenated abstracts [dico in which keys=author_IDs & values=concatenated abstracts as a string].
Dictionary name: DictForAuthor
Dictionary saved as: authors_with_all_abstracts.pkl
'''
# print("extracting abstracts")
# DictOfAbstracts = {}
# infile = open('abstracts_data.pkl', 'rb')
# DictOfAbstracts = pickle.load(infile)
# infile.close()
# print("extracting authors-paperIDs [dico]")
# DictOfPaperID = process_authorFiles()
# # print(DictOfPaperID['2318996053']) # works
# print("matching authors to all of their abstracts [dico]")
# DictForAuthor = dictionary_concatenation(DictOfAbstracts, DictOfPaperID)
# # print(DictForAuthor['2318996053']) # works
# with open('authors_with_all_abstracts.pkl', 'wb') as f:
#     pickle.dump(DictForAuthor, f)
# f.close()
# print("Operation completed. Check authors_with_all_abstracts.pkl file.")


In [ ]:
## don't run ##
''' 
To generate a dictionary matching the authors to their *separated* abstracts [dico in which keys=author_IDs & values=a list with the separated abstracts].
Dictionary name: DictForAuthor2
Dictionary saved as: authors_with_all_abstracts_separated.pkl
'''
print("extracting abstracts")
DictOfAbstracts = {}
infile = open('abstracts_data.pkl', 'rb')
DictOfAbstracts = pickle.load(infile)
infile.close()

print("extracting authors-paperIDs [dico]")
DictOfPaperID = process_authorFiles()
print(DictOfPaperID['2318996053'])

print("matching authors to all of their abstracts *separated* [dico-list]")
DictForAuthor = dictionary_concatenation2(DictOfAbstracts, DictOfPaperID) 
print(DictForAuthor['2318996053'])
print("len(DictForAuthor['2318996053']) = ", len(DictForAuthor['2318996053']))

with open('authors_with_all_abstracts_separated.pkl', 'wb') as f:
    pickle.dump(DictForAuthor, f)
f.close()
print("Operation completed. Check authors_with_all_abstracts_separated.pkl file.")


In [ ]:
## don't run ##
# # takes care of cleaning the stringremove stopwords
''' 
To generate a dictionary matching the authors to their *separated & cleaned* abstracts 
[dico in which keys=author_IDs & values=a list with the separated abstracts cleaned and w/o stopwords].
Dictionary name: DictForAuthor_cleaned
Dictionary saved as: authors_with_all_abstracts_separated_cleaned.pkl
'''
DictForAuthor_cleaned = DictForAuthor # make copy
specialChars = ".,()"
specialChars2 = ['\r', '\n', '\t', '\\r', '\\n', '\\t', '\\r\\n\\r\\n', '\\r\\n\\r\\n']
stop_words = {'ourselves', 'hers', 'between', 'yourself', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'itself', 'other', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while',
              'above', 'up', 'to', 'ours', 'had', 'she', 'when', 'at', 'any', 'before', 'them', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}

for authorID in tqdm(DictForAuthor_cleaned): # iterate over dictionary
    for i in range(len(DictForAuthor_cleaned[authorID])): # iterate over the author's separate abstracts
        abstractText = str(DictForAuthor_cleaned[authorID][i])
        ## clean string and remove stopwords ##
        for specialChar in specialChars:
            abstractText = abstractText.replace(specialChar, "")
        for specialChar2 in specialChars2:
            abstractText = abstractText.replace(specialChar2, "")
        abstractText = abstractText.replace("   ", " ")
        abstractText = abstractText.replace("  ", " ")
        abstractText = abstractText.strip()
        abstractText_words = abstractText.split()  # isolate words
        for j in range(len(abstractText_words)):
            if abstractText_words[j] in stop_words:
                abstractText_words[j] = ""
        # for stop_word in stop_words:
        #     abstractText = abstractText.replace(stop_word, "")

        temp = ' '.join(abstractText_words)
        temp = temp.strip()
        temp = temp.replace("  ", " ")
        temp = temp.replace("   ", " ")
        DictForAuthor_cleaned[authorID][i] = ' '.join(temp.split())

with open('authors_with_all_abstracts_separated_cleaned.pkl', 'wb') as f:
    pickle.dump(DictForAuthor_cleaned, f)
f.close()
print("Operation completed. Check authors_with_all_abstracts_separated_cleaned.pkl file.")


Can continue here ---------- directly.

In [2]:
# authors_with_all_abstracts_separated_cleaned.pkl is GOOD.
DicoCleaned = {}
infile = open('authors_with_all_abstracts_separated_cleaned.pkl', 'rb')
DicoCleaned = pickle.load(infile)
infile.close()
print(len(DicoCleaned))


217801


In [4]:
# checkClean = []
for author in DicoCleaned:
    allAbstracts = ' '.join(DicoCleaned[author])
    checkClean = allAbstracts.split()
    if "The" in checkClean:
        print("The present")
        print(author)
        print(DicoCleaned[author])
        break


The present
1036332
['An underground utility conveyance 10 may precisely located impressing both locating tone confirmation tone conveyance The locating tone RF signal produces first field radiates earth sensed first above-ground signal detector 22 establish general location conveyance The confirmation tone near-DC signal produces second field relatively limited radiation path A second detector 24 inserted earth successive places region conveyance located establish precise location', 'The invention relates method system wireless network based mapping method system supporting field personnel access select download facilities related maps related information Mapping files manipulated accordance grid block overlay first level map display Each grid blocks uniquely identifiable indexed one algorithms necessary local creation enhanced and/or enlarged second level display', 'The present invention system method searching large database dig location tickets tickets particular interest A user lo

In [16]:
count = 0
for author in DicoCleaned:
    print(author)
    count +=1
    if count == 10:
        break
## order is preserved (i.e. same as in author_papers.txt) ##

1036332
1101850
1336878
1515524
1606427
2728936
3720290
4265922
4631324
4652436


In [7]:
example1_BERTembedding = Embed_Author(example1_concat)
print("done")

Vectorizing batch0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (736 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (736) must match the size of tensor b (512) at non-singleton dimension 1

In [4]:
## do not run ##
def process_smart(dico_inner, method, minNumWords=450):
    numOps = 0
    new_dico = {}
    if method == "endOf":
        for authorID in tqdm(dico_inner):
            # print(authorID)
            abstractConcat = ' '.join(dico_inner[authorID])
            numAbstracts = len(dico_inner[authorID]); abstractLength = len(abstractConcat); numWords = len(abstractConcat.split())
            # print("numAbstracts : ", numAbstracts); print("abstractLength : ", abstractLength); print("numWords : ", numWords)
            if numWords < minNumWords:
                continue
            numOps += 1
            diff = numWords - minNumWords; numRemovalsPerAbstract = diff // numAbstracts; numAdditionalRemovals = diff % numAbstracts
            # print("numRemovalsPerAbstract : ", numRemovalsPerAbstract); print("numAdditionalRemovals : ", numAdditionalRemovals)
            for i in range(numAbstracts): ## for each abstract rmv words
                abstract = dico_inner[authorID][i]; abstractWords = abstract.split(); size = len(abstractWords); 
                abstractWords = abstractWords[:size-numRemovalsPerAbstract]; dico_inner[authorID][i] = ' '.join(abstractWords)
            for j in range(numAdditionalRemovals):
                abstract = dico_inner[authorID][numAbstracts-1-j]; abstractWords = abstract.split(); size = len(abstractWords); 
                abstractWords = abstractWords[:size-1]; dico_inner[authorID][numAbstracts-1-j] = ' '.join(abstractWords)

            abstractConcat = ' '.join(dico_inner[authorID])
            abstractLength = len(abstractConcat)
            numWords = len(abstractConcat.split())
            # print("abstractLength : ", abstractLength); print("numWords : ", numWords)
            new_dico = dico_inner

    if method == "random":
        print("random method used")
        for authorID in tqdm(dico_inner):
            abstractConcat = ' '.join(dico_inner[authorID])
            abstractConcatWord = abstractConcat.split(); numWords = len(abstractConcatWord)
            if numWords <= minNumWords:
                new_dico[authorID] = DicoCleaned[authorID]
                continue
            else:
                numOps += 1
                while(numWords > minNumWords):
                    rmv_index = random.randint(0, numWords-1)
                    del abstractConcatWord[rmv_index]
                    numWords -= 1
                new_dico[authorID] = ' '.join(abstractConcatWord)

    print("Number of authors-abstracts reduced: ", numOps)

    return new_dico



# dico2 = dicoGame.copy()
dico_new = process_smart(DicoCleaned, "random")


random method used


100%|██████████| 217801/217801 [00:23<00:00, 9230.85it/s]

Number of authors-abstracts reduced:  85328


In [ ]:
## do not run ##
## works/good##
sentences_cleaned = []
for author in DicoCleaned:
    sentences_cleaned.append(' '.join(DicoCleaned[author]))

textfile = open("sentences_cleaned.txt", "w")
for element in sentences_cleaned:
    textfile.write(element + "\n")
textfile.close()


In [13]:
print(len(DicoCleaned))
print("random method used")
minNumWords = 450
new_dico = {}
for authorID in tqdm(DicoCleaned):
    abstractConcat = ' '.join(DicoCleaned[authorID])
    abstractConcatWord = abstractConcat.split()
    numWords = len(abstractConcatWord)
    # print(numWords)
    
    if numWords <= minNumWords:
        new_dico[authorID] = ' '.join(abstractConcatWord)
        continue
    else:
        while(numWords > minNumWords):
            rmv_index = random.randint(0, numWords-1)
            del abstractConcatWord[rmv_index]
            numWords -= 1
        # print("' '.join(abstractConcatWord) : \n", ' '.join(abstractConcatWord), "\n")
        new_dico[authorID] = ' '.join(abstractConcatWord)
        if len(new_dico[authorID].split()) > minNumWords:
            print("WOW")
            print(authorID)
            break

print(len(new_dico))


217801
random method used


100%|██████████| 217801/217801 [00:55<00:00, 3927.19it/s]

217801


In [14]:
for author in new_dico:
    abstr = new_dico[author].split()
    # print(abstr)
    if len(abstr) > 450:
        print("GODDAMN")


In [15]:
with open('authors_with_all_abstracts_random_max450words.pkl', 'wb') as f:
    pickle.dump(new_dico, f)
f.close()
print("Operation completed. Check authors_with_all_abstracts_random_max450words.pkl file.")


Operation completed. Check authors_with_all_abstracts_random_max450words.pkl file.


In [16]:
### worked ###
sentences_cleaned_450words = [] 
for author in new_dico:
    sentences_cleaned_450words.append(new_dico[author])

with open('sentences_cleaned_random_max450words.pkl', 'wb') as f:
    pickle.dump(sentences_cleaned_450words, f)
f.close()

# textfile = open("sentences_cleaned_random_max450words.txt", "w")
# for element in sentences_cleaned_450words:
#     textfile.write(element + "\n")
# textfile.close()


In [17]:
infile = open('sentences_cleaned_random_max450words.pkl', 'rb')
sentencesX = pickle.load(infile)
infile.close()
print(len(sentencesX))
# print(len(new_dico))

print(sentencesX[20])
print()
# print(new_dico['7769909'])


217801
mobile cloud rapidly terms use case functionalityBoth two rapidly case This reviews current energy consumption cloud computing proposes system whereby applications may profiled consumption augmentation required negotiate external optimum Such system useful resources may computing functionality location computing provisioned both end users These two fields order greater functionality mobile devices number different ways Augmentation resources cloud shown one energy power mobile devices due resource constrained communication interfaces often fine line offloading resources The internet things IoT current state large number connected huge amount data needs lot operations useful information control data requires design accelerate enhance The defined new paradigm appeared hide all complexity system all controls management devices things inside middleware layer software In work defined model simplify IoT management process vital solution challenges architecture store produced IoT softw

In [19]:
subset = sentencesX[0:6000]
print(len(subset))
BERTembedding = Embed_Author(subset)
print("done")


6000
Vectorizing batch0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (700) must match the size of tensor b (512) at non-singleton dimension 1

In [15]:
infile = open('authors_with_all_abstracts_separated_cleaned.pkl', 'rb')
dicoY = pickle.load(infile)
infile.close()
print(dicoY['7769909'])


['The increasing pervasiveness location-acquisition technologies GPS GSM networks etc leading collection large spatio-temporal datasets opportunity discovering usable knowledge movement behaviour fosters novel applications services In paper move towards direction develop extension sequential pattern mining paradigm analyzes trajectories moving objects We introduce trajectory patterns concise descriptions frequent behaviours terms both space ie regions space visited movements time ie duration movements In setting provide general formal statement novel mining problem study several different instantiations different complexity The various approaches empirically evaluated real data synthetic benchmarks comparing strengths weaknesses', 'Our understanding individual mobility patterns shape impact social network limited but essential deeper understanding network dynamics evolution This question largely unexplored partly due difficulty obtaining large-scale society-wide data simultaneously cap

In [24]:
infile = open('fullEmbeddings_random.pkl', 'rb')
fullEmbeddings_random = pickle.load(infile)
infile.close()
print(len(fullEmbeddings_random))


37
